In [159]:
import pandas as pd
import numpy as np
from tqdm import tqdm, trange

data = pd.read_csv("ner_dataset.csv", encoding="latin1").fillna(method="ffill")
data.tail(10)

,Sentence #,Word,POS,Tag
1048565,Sentence: 47958,impact,NN,O
1048566,Sentence: 47958,.,.,O
1048567,Sentence: 47959,Indian,JJ,B-gpe
1048568,Sentence: 47959,forces,NNS,O
1048569,Sentence: 47959,said,VBD,O
1048570,Sentence: 47959,they,PRP,O
1048571,Sentence: 47959,responded,VBD,O
1048572,Sentence: 47959,to,TO,O
1048573,Sentence: 47959,the,DT,O
1048574,Sentence: 47959,attack,NN,O


In [160]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [161]:
getter = SentenceGetter(data)

In [162]:
sentences = [" ".join([s[0] for s in sent]) for sent in getter.sentences]
sentences[0]

'Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .'

In [166]:
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam

In [167]:
MAX_LEN = 75 # maximum number of tokens in a sequence
bs = 32 # batch size

In [168]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [169]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [256]:
labels = [[s[2] for s in sent] for sent in getter.sentences]

In [257]:
# tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
tokenize_text = []
for sent in range(len(sentences)):
    tokenize_text.append([])
    splitted = sentences[sent].split()
    for word in range(len(splitted)):
        for padding in range(len(tokenizer.tokenize(splitted[word])) - 1):
            labels[sent].insert(len(tokenize_text[-1])+padding+1,'X')
        tokenize_text[-1].extend(tokenizer.tokenize(splitted[word]))

In [258]:
checking = np.random.randint(len(tokenized_texts))

print(sentences[checking],"\n")
print(tokenized_texts[checking],'\n')
print("Length of tokenized text: %d\n"%len(tokenized_texts[checking]))
print(labels[checking],'\n')
print("Length of labels: %d\n"%len(labels[checking]))
for token in range(len(tokenized_texts[checking])):
    print(tokenized_texts[checking][token]+":"+labels[checking][token])

A spokeswoman for Kennedy Monday said it is not clear when his doctor will release information about his condition . 

['a', 'spoke', '##sw', '##oman', 'for', 'kennedy', 'monday', 'said', 'it', 'is', 'not', 'clear', 'when', 'his', 'doctor', 'will', 'release', 'information', 'about', 'his', 'condition', '.'] 

Length of tokenized text: 22

['O', 'O', 'X', 'X', 'O', 'B-org', 'B-tim', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'] 

Length of labels: 22

a:O
spoke:O
##sw:X
##oman:X
for:O
kennedy:B-org
monday:B-tim
said:O
it:O
is:O
not:O
clear:O
when:O
his:O
doctor:O
will:O
release:O
information:O
about:O
his:O
condition:O
.:O


In [259]:
tags_vals = list(set(data["Tag"].values))
tags_vals.append('X')
tag2idx = {t: i for i, t in enumerate(tags_vals)}
print(tag2idx)

{'B-geo': 0, 'I-nat': 1, 'B-per': 2, 'I-org': 3, 'I-eve': 4, 'I-tim': 5, 'I-per': 6, 'I-gpe': 7, 'B-gpe': 8, 'B-tim': 9, 'B-eve': 10, 'O': 11, 'I-geo': 12, 'B-art': 13, 'B-org': 14, 'I-art': 15, 'B-nat': 16, 'X': 17}


In [260]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [261]:
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["O"], padding="post",
                     dtype="long", truncating="post")

In [262]:
attention_masks = [[float(i>0) for i in ii] for ii in input_ids]

In [263]:
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags, 
                                                            random_state=2019, test_size=0.1)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2019, test_size=0.1)

In [264]:
#random checking

print(tag2idx,"\n")
checking = np.random.randint(tr_inputs.shape[0])
decoded_tokens = tokenizer.convert_ids_to_tokens(np.array((tr_inputs[checking])))
decoded_tokens = [word for word in decoded_tokens if word != '[PAD]']
print(decoded_tokens,"\n")
for word in range(len(decoded_tokens)):
    print("%s: %s "%(decoded_tokens[word],np.int(tr_tags[checking][word])))

{'B-geo': 0, 'I-nat': 1, 'B-per': 2, 'I-org': 3, 'I-eve': 4, 'I-tim': 5, 'I-per': 6, 'I-gpe': 7, 'B-gpe': 8, 'B-tim': 9, 'B-eve': 10, 'O': 11, 'I-geo': 12, 'B-art': 13, 'B-org': 14, 'I-art': 15, 'B-nat': 16, 'X': 17} 

['china', "'", 's', 'agriculture', 'ministry', 'also', 'confirmed', 'monday', 'that', 'bird', 'flu', 'was', 'the', 'cause', 'of', 'illness', 'among', 'chickens', ',', 'ducks', ',', 'and', 'geese', 'last', 'week', 'in', 'the', 'inner', 'mongolia', 'autonomous', 'region', 'and', 'central', 'china', "'", 's', 'hub', '##ei', 'province', '.'] 

china: 0 
': 11 
s: 17 
agriculture: 14 
ministry: 3 
also: 11 
confirmed: 11 
monday: 9 
that: 11 
bird: 11 
flu: 11 
was: 11 
the: 11 
cause: 11 
of: 11 
illness: 11 
among: 11 
chickens: 11 
,: 11 
ducks: 11 
,: 11 
and: 11 
geese: 11 
last: 11 
week: 11 
in: 11 
the: 11 
inner: 14 
mongolia: 3 
autonomous: 3 
region: 3 
and: 11 
central: 11 
china: 0 
': 11 
s: 17 
hub: 11 
##ei: 17 
province: 11 
.: 11 


In [265]:
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [266]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

In [267]:
model = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=len(tag2idx))

In [19]:
# model.cuda()

In [268]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)

In [269]:
from seqeval.metrics import f1_score

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
epochs = 5
max_grad_norm = 1.0

for _ in trange(epochs, desc="Epoch"):
    # TRAIN loop
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        # forward pass
        loss = model(b_input_ids, token_type_ids=None,
                     attention_mask=b_input_mask, labels=b_labels)
        # backward pass
        loss.backward()
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        model.zero_grad()
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))
    # VALIDATION on validation set
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in valid_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():
            tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                                  attention_mask=b_input_mask, labels=b_labels)
            logits = model(b_input_ids, token_type_ids=None,
                           attention_mask=b_input_mask)
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.append(label_ids)
        
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        eval_loss += tmp_eval_loss.mean().item()
        eval_accuracy += tmp_eval_accuracy
        
        nb_eval_examples += b_input_ids.size(0)
        nb_eval_steps += 1
    eval_loss = eval_loss/nb_eval_steps
    print("Validation loss: {}".format(eval_loss))
    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
    pred_tags = [tags_vals[p_i] for p in predictions for p_i in p]
    valid_tags = [tags_vals[l_ii] for l in true_labels for l_i in l for l_ii in l_i]
    print("F1-Score: {}".format(f1_score(pred_tags, valid_tags)))

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
model.eval()
predictions = []
true_labels = []
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
for batch in valid_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
        tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                              attention_mask=b_input_mask, labels=b_labels)
        logits = model(b_input_ids, token_type_ids=None,
                       attention_mask=b_input_mask)
        
    logits = logits.detach().cpu().numpy()
    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
    label_ids = b_labels.to('cpu').numpy()
    true_labels.append(label_ids)
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)

    eval_loss += tmp_eval_loss.mean().item()
    eval_accuracy += tmp_eval_accuracy

    nb_eval_examples += b_input_ids.size(0)
    nb_eval_steps += 1

pred_tags = [[tags_vals[p_i] for p_i in p] for p in predictions]
valid_tags = [[tags_vals[l_ii] for l_ii in l_i] for l in true_labels for l_i in l ]
print("Validation loss: {}".format(eval_loss/nb_eval_steps))
print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
print("Validation F1-Score: {}".format(f1_score(pred_tags, valid_tags)))